In [22]:
num_sample = 300
num_burn = 20
sample_size = 800
n_cpu = 10

min_degree = 1
max_degree = 1

In [23]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1,2,0.1,2,0.1,-2,0.1,2,0.1,0])

Y_chain, A_chain, L_chain = sample_network_chain(adj1, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

1 9


100%|██████████| 320/320 [00:00<00:00, 377.05it/s]


(0.6088388224640909,
 0.4185750000000001,
 0.5322125,
 0.34354810870108954,
 0.370410925858805)

In [24]:
res_truth1 = agcEffect(
    adj1,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=100,
    burnin_R=100,
    seed=0
)
res_truth2 = agcEffect(
    adj1,
    tau, rho, nu, beta,
    treatment_allocation=0.3,
    R=100,
    burnin_R=100,
    seed=0
)

ground_truth1 = {}
ground_truth1['average'] = res_truth1['average']
ground_truth1['direct'] = res_truth1['direct_effect']
ground_truth1['indirect'] = res_truth1['psi_0_gamma'] - res_truth2['psi_0_gamma']
ground_truth1['spillover_effect'] = res_truth1['spillover_effect']
ground_truth1['psi_1_gamma'] = res_truth1['psi_1_gamma']
ground_truth1['psi_0_gamma'] = res_truth1['psi_0_gamma']
ground_truth1['psi_zero'] = res_truth1['psi_zero']

ground_truth1

100%|██████████| 200/200 [00:00<00:00, 593.33it/s]


{'average': 0.6392625000000001,
 'direct': 0.30823750000000005,
 'indirect': 0.011837499999999945,
 'spillover_effect': 0.02166250000000014,
 'psi_1_gamma': 0.7344250000000001,
 'psi_0_gamma': 0.42618750000000005,
 'psi_zero': 0.4045249999999999}

In [25]:
from utils import run_pll
from run_pll import run_dr_em_raw

treat_p = 0.7
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

Multiprocessing <function run_dr_em_raw at 0x30cb68ee0> in 100 tasks, with 10 processes...


/Users/ljzljz/Documents/DR/dr-chain-graph/drnet_em.py:184: RuntimeWarning: divide by zero encountered in divide
  psi_vec[:,a,g] = beta_hat + I / pi * (Y - beta_hat)
/Users/ljzljz/Documents/DR/dr-chain-graph/drnet_em.py:184: RuntimeWarning: divide by zero encountered in divide
  psi_vec[:,a,g] = beta_hat + I / pi * (Y - beta_hat)
/Users/ljzljz/Documents/DR/dr-chain-graph/drnet_em.py:184: RuntimeWarning: divide by zero encountered in divide
  psi_vec[:,a,g] = beta_hat + I / pi * (Y - beta_hat)
/Users/ljzljz/Documents/DR/dr-chain-graph/drnet_em.py:184: RuntimeWarning: divide by zero encountered in divide
  psi_vec[:,a,g] = beta_hat + I / pi * (Y - beta_hat)
/Users/ljzljz/Documents/DR/dr-chain-graph/drnet_em.py:184: RuntimeWarning: invalid value encountered in multiply
  psi_vec[:,a,g] = beta_hat + I / pi * (Y - beta_hat)
/Users/ljzljz/Documents/DR/dr-chain-graph/drnet_em.py:184: RuntimeWarning: divide by zero encountered in divide
  psi_vec[:,a,g] = beta_hat + I / pi * (Y - beta_hat)
/Us

Multiprocessing finished.


In [26]:
from utils import run_pll
from run_pll import run_dr_em_raw

treat_p = 0.3
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

Multiprocessing <function run_dr_em_raw at 0x30cb68ee0> in 100 tasks, with 10 processes...


/Users/ljzljz/Documents/DR/dr-chain-graph/drnet_em.py:184: RuntimeWarning: divide by zero encountered in divide
  psi_vec[:,a,g] = beta_hat + I / pi * (Y - beta_hat)
/Users/ljzljz/Documents/DR/dr-chain-graph/drnet_em.py:184: RuntimeWarning: divide by zero encountered in divide
  psi_vec[:,a,g] = beta_hat + I / pi * (Y - beta_hat)
/Users/ljzljz/Documents/DR/dr-chain-graph/drnet_em.py:184: RuntimeWarning: divide by zero encountered in divide
  psi_vec[:,a,g] = beta_hat + I / pi * (Y - beta_hat)
/Users/ljzljz/Documents/DR/dr-chain-graph/drnet_em.py:184: RuntimeWarning: divide by zero encountered in divide
  psi_vec[:,a,g] = beta_hat + I / pi * (Y - beta_hat)
/Users/ljzljz/Documents/DR/dr-chain-graph/drnet_em.py:184: RuntimeWarning: invalid value encountered in multiply
  psi_vec[:,a,g] = beta_hat + I / pi * (Y - beta_hat)
/Users/ljzljz/Documents/DR/dr-chain-graph/drnet_em.py:184: RuntimeWarning: divide by zero encountered in divide
  psi_vec[:,a,g] = beta_hat + I / pi * (Y - beta_hat)
/Us

Multiprocessing finished.


In [27]:
# read results
import numpy as np
from utils import *


results1 = np.load(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

  3%|▎         | 3/100 [00:00<00:05, 18.32it/s]/Users/ljzljz/Documents/DR/dr-chain-graph/utils.py:55: RuntimeWarning: invalid value encountered in subtract
  res_spillover = res_list_array_dr1[i,:,cols_raw.index('psi_0_gamma')] - res_list_array_dr1[i,:,cols_raw.index('psi_zero')]
/Users/ljzljz/Documents/DR/dr-chain-graph/drnet.py:305: RuntimeWarning: invalid value encountered in subtract
  g = psi_vec - avg_effects
/Users/ljzljz/Documents/DR/dr-chain-graph/drnet.py:316: RuntimeWarning: invalid value encountered in double_scalars
  hac_var += weight * g[i] * g[j]
 12%|█▏        | 12/100 [00:00<00:03, 22.43it/s]/opt/anaconda3/envs/ljz/lib/python3.10/site-packages/numpy/core/_methods.py:180: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
100%|██████████| 100/100 [00:07<00:00, 13.05it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.68, 0.72, 0.62, 0.63, 0.8 , 0.81, 0.75]),
 'bias': array([nan, nan, nan, nan, nan, nan, nan]),
 'mse': array([nan, nan, nan, nan, nan, nan, nan]),
 'var': array([nan, nan, nan, nan, nan, nan, nan]),
 'ci_length': array([nan, nan, nan, nan, nan, nan, nan]),
 'true_effect': array([0.6392625, 0.3082375, 0.0118375, 0.0216625, 0.734425 , 0.4261875,
        0.404525 ])}

In [28]:
from tqdm import tqdm
from drnet import compute_avg_effects_std_from_raw

results1 = np.load(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}.npy', allow_pickle=True)

var = []
var_sim = []
est = []
idx = 3
for i in tqdm(range(Y_chain.shape[0])):
    avg_effects, se_hac = compute_avg_effects_std_from_raw(results1[i,:,idx],adj1,h=3)
    var.append(se_hac**2*sample_size)
    var_sim.append(np.var(results1[i,:,idx]))
    est.append(avg_effects)

var_true = np.var(est)
print(np.mean(var), np.mean(var_sim), var_true*sample_size)

  0%|          | 0/100 [00:00<?, ?it/s]/opt/anaconda3/envs/ljz/lib/python3.10/site-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
100%|██████████| 100/100 [00:01<00:00, 57.39it/s]

nan nan nan
